In [9]:
import neo4j
import osmnx as ox
import pandas as pd

In [10]:
pd.set_option('display.max_colwidth', None)

In [2]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [6]:
# Define the area of interest
place = "Hamburg, Deutschland"

# Define the tags for different POIs you're interested in
tags = {
    # Additional categories
    'building': ['civic', 'public'],
}

In [23]:
# Fetch the POIs
pois = ox.features_from_place(place, tags=tags)

# Display first few rows to check the data
pois.columns.to_list()

['geometry',
 'addr:city',
 'addr:housenumber',
 'addr:postcode',
 'addr:street',
 'building',
 'name',
 'office',
 'phone',
 'website',
 'wheelchair',
 'wheelchair:description',
 'wikidata',
 'addr:country',
 'amenity',
 'check_date',
 'check_date:opening_hours',
 'community_centre',
 'community_centre:for',
 'description:de',
 'opening_hours',
 'operator',
 'social_facility',
 'social_facility:for',
 'fee',
 'leisure',
 'sport',
 'description',
 'short_name',
 'surveillance',
 'wikipedia',
 'building:levels',
 'old_name',
 'roof:shape',
 'roof:levels',
 'source',
 'note',
 'architect',
 'construction_date',
 'dsa:criteria',
 'heritage',
 'heritage:operator',
 'historic',
 'image',
 'ref:dsa',
 'contact:website',
 'opening_hours:signed',
 'opening_hours:url',
 'alt_name',
 'contact:email',
 'contact:fax',
 'contact:phone',
 'layer',
 'operator:wikidata',
 'operator:wikipedia',
 'police',
 'official_name',
 'height',
 'emergency',
 'ref',
 'short_name:de',
 'government',
 'operator:typ

In [20]:
pois.type.value_counts()

Polygon    354
Point        4
Name: count, dtype: int64

In [25]:
pois_polygons = pois[pois.type == 'Polygon']

In [28]:
pois_polygons[['geometry', 'name', 'addr:street']].head()

geometry  \
element  id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
relation 308234                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [ ]:
# Funktion, um zu entscheiden, ob ein Polygon durch seinen Centroid ersetzt werden soll
def simplify_to_centroid_if_small(geometry, max_area_threshold=0.0000679123):  # Schwellenwert anpassen
    """
    Ersetzt ein Polygon durch seinen Centroid, wenn die Fläche unter dem Schwellenwert liegt.
    Der Schwellenwert ist in Quadratgrad (für WGS84). 
    0.0001 entspricht ungefähr 1 km² in Hamburg.
    0.0000679123 entspricht ungefähr 0,5 km² in Hamburg.
    """
    if geometry.geom_type in ['Polygon', 'MultiPolygon']:
        area = geometry.area
        if area < max_area_threshold:
            return geometry.centroid
    return None

# Anwenden auf deine POIs DataFrame
pois_polygons['simplified_geometry'] = pois_polygons['geometry'].apply(simplify_to_centroid_if_small)

c:\Users\paul-\anaconda3\envs\py310\lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [31]:
pois_polygons[['geometry', 'simplified_geometry', 'name', 'addr:street']].head()

geometry  \
element  id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
relation 308234                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [34]:
pois_polygons[pois_polygons.simplified_geometry == None][['geometry', 'simplified_geometry', 'name', 'addr:street']].head()

,,geometry,simplified_geometry,name,addr:street
element,id,,,,


In [13]:
pois.columns.to_list()

['geometry',
 'addr:city',
 'addr:housenumber',
 'addr:postcode',
 'addr:street',
 'building',
 'name',
 'office',
 'phone',
 'website',
 'wheelchair',
 'wheelchair:description',
 'wikidata',
 'addr:country',
 'amenity',
 'check_date',
 'check_date:opening_hours',
 'community_centre',
 'community_centre:for',
 'description:de',
 'opening_hours',
 'operator',
 'social_facility',
 'social_facility:for',
 'fee',
 'leisure',
 'sport',
 'description',
 'short_name',
 'surveillance',
 'wikipedia',
 'building:levels',
 'old_name',
 'roof:shape',
 'roof:levels',
 'source',
 'note',
 'architect',
 'construction_date',
 'dsa:criteria',
 'heritage',
 'heritage:operator',
 'historic',
 'image',
 'ref:dsa',
 'contact:website',
 'opening_hours:signed',
 'opening_hours:url',
 'alt_name',
 'contact:email',
 'contact:fax',
 'contact:phone',
 'layer',
 'operator:wikidata',
 'operator:wikipedia',
 'police',
 'official_name',
 'height',
 'emergency',
 'ref',
 'short_name:de',
 'government',
 'operator:typ